# Lab 2.8: Tree Based Methods

In this case, we will be conduct a simpler exercise with decision trees using previous implementations. In particular, we will make use of several implemented methods in ML libraries s.a. `sklearn` (_that should be good news for you, doesn't it?_). With this, we will try to explore the main characteristics of decision trees, that you will also have to explore in the more theoretical part of the lab (the other exercise, the one on the pdf).

We will begin, as usual, importing the relevant libraries:

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import mode

import seaborn as sns


# ML libraries to construct, use and analyse the trees
from sklearn.tree import DecisionTreeRegressor, export_graphviz, DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier
from sklearn.model_selection import train_test_split
from io import StringIO
from IPython.display import Image  
from sklearn.metrics import mean_squared_error, confusion_matrix
import pydotplus

from sklearn.metrics import mean_squared_error

ModuleNotFoundError: No module named 'pydotplus'

We will first apply this to a regression dataset so that you see the way the model is constructed for this case 

## Regression Tree

For starters, let us try out a regression tree. To that end, first load the `Hitters.csv` dataset from the `data` 
folder.

In [2]:
#(make sure you remove the None values!)
hitters  = pd.read_csv('../data/Hitters.csv', sep=',')
hitters = hitters.dropna()

# Print the columns here to check their names
print(hitters.columns)

Index(['Player', 'AtBat', 'Hits', 'HmRun', 'Runs', 'RBI', 'Walks', 'Years',
       'CAtBat', 'CHits', 'CHmRun', 'CRuns', 'CRBI', 'CWalks', 'League',
       'Division', 'PutOuts', 'Assists', 'Errors', 'Salary', 'NewLeague'],
      dtype='object')


For this first case, we will only use the variables `Years` and `Hits` for the tree. Our target variable will be `Salary`. Separate them into `X` and `y`

In [3]:
# Separate the independent variables (X) from the dependent one (y - salary)
 
X = hitters[['Years', 'Hits']]
y = hitters['Salary']

Construct a _decision tree regressor_ using the `sklearn` function and fit it. To do that, check out the `DecisionTreeRegressor` in sklearn and implement it here.

For reproducibility, fix the `random_state` to `0` and the `max_leaf_nodes` to `3` (make sure you know what this last thing does!)

In [4]:
# Construct the regressor
regressor = regressor = DecisionTreeRegressor(random_state=0, max_leaf_nodes=3)

# Fit it with the .fit method
regressor.fit(X, y)  

DecisionTreeRegressor(max_leaf_nodes=3, random_state=0)

Now we will employ some functions engrained in `StringIO` alongside the method `export_graphviz` from the `tree` object in sklearn. This will enable us to visualize the constructed tree.

In [5]:
dot_data = StringIO()
export_graphviz(regressor, 
                out_file=dot_data, 
                feature_names=['Years', 'Hits'], 
                filled=True, 
                class_names=None)

graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
Image(graph.create_png())

NameError: name 'pydotplus' is not defined

>Question: Describe the previous tree. What do you see? Do you think this will work well? <br><br>
Nuestro arbol cuenta tres nodos hojas obtenidos tras las condiciones de Years <= 4.5 y Hits <=117.5. Al tratarse de un arbol muy simple, no resultará útil para modelos complejos


Now we will plot the decision regions using the information on the cuts. Add lines wherever needed so that you can see the decision boundaries for the regression tree above

In [ ]:
hitters.plot('Years', 'Hits', kind='scatter', color='orange', figsize=(7,6))
plt.xlim(0,25)
plt.ylim(ymin=-5)

# Add whatever you may need here to clearly plot the decision boundaries 
plt.axvline(x=4.5, color='blue', linestyle='--', label='Years = 4.5')
plt.axhline(y=117.5, xmin=4.5/25, xmax=1, color='red', linestyle='--', label='Hits = 117.5')
plt.legend()
plt.show()

## Tree Size

Now, for the previous part we limited the growth of the tree so that we recovered a simple (but easily interpretable) tree. Now we will go all-out: we will construct a more exhaustive tree using different variables. For this particular case, let us use **all variables except** `League`, `Division`, `NewLeague` and `Salary` as independent variables to predict, precisely, the `Salary` value. 

In [ ]:
# Construct the input variable dataset
X = hitters[['AtBat', 'Hits', 'HmRun', 'Runs', 'RBI', 'Walks', 'Years',
       'CAtBat', 'CHits', 'CHmRun', 'CRuns', 'CRBI', 'CWalks',
        'PutOuts', 'Assists', 'Errors']]

# Print the column names to check
print(X.columns)

We will now perform the train/test split, but we will do it so that the proportion of train and test examples is $50\%$ (that is, train and test consist on $50\%$ of the datapoints).

In [ ]:
# Perform the train-test split here. Do it so that the  
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.5, train_size=0.5, random_state=0) #Fill the NAs, fixing also the random_state to 0 for reproducibility

Now, train the tree to its fullest extent: put no limits on the growth and see what happens. You can re-use some of the previous `graphviz` code to visualize the tree here. Plot the complete tree.

In [ ]:
# Train the tree without limits to its growth (random_state = 0)
unlimited_tree_regressor =DecisionTreeRegressor(random_state=0)
unlimited_tree_regressor.fit(x_train, y_train) 

Reuse the code you need to plot the tree here

In [ ]:

dot_data = StringIO()
export_graphviz(unlimited_tree_regressor, 
                out_file=dot_data, 
                feature_names=X.columns, 
                filled=True, 
                class_names=None)

graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
Image(graph.create_png())


> Question: What do you see? What can you say about this tree? Does it have any important properties? <br><br>
El árbol ha crecido completamente, alcanzando un punto en el que todos sus nodos terminales son completamente puros (coste = 0). Esto sugiere un posible sobreajuste a los datos, ya que las divisiones se han realizado hasta clasificar perfectamente cada caso. Como resultado, es probable que el modelo no tenga un buen desempeño al aplicarse a nuevos datos, ya que su capacidad de generalización se ve afectada.

Maybe we went _a bit too far_ with the tree... Let's set up some limitations to see everything better. Try setting the `max_features` to 9, and the `max_depth` to 4. (_It is important you understand what these parameters do! Check out the documentation in the [library](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html)_)

In [ ]:
regressor_2 =  DecisionTreeRegressor(max_features=9, random_state=0, max_depth=4)
regressor_2.fit(x_train, y_train) 

Reuse the code you may need to plot the tree here

In [ ]:
dot_data = StringIO()
export_graphviz(regressor_2, 
                out_file=dot_data, 
                feature_names= x_train.columns, 
                filled=True, 
                class_names=None)

graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
Image(graph.create_png())

Let us see if this tree works well at all... Since we are performing regression, we can obtain the RMSE (we use the Root MSE since it shares the same dimensions of the outputs):

In [ ]:
predictions = regressor_2.predict(x_test)
mse = mean_squared_error(y_test, predictions)
rmse = np.sqrt(mse)
# Print the RMSE for the predictions
print(f"Root Mean Squared Error (RMSE): {rmse}")

Now, the question is: how do we know which tree depth to select here? As you may expect, the answer is, as almost always here, performing _cross validation_. In this particular instance we will not conduct exhaustive cross validation. Instead, we will do it in a very simple manner, obtaining *a single tree* for each depth value we want, fitting it to the data and seeing how well does it perform both in train and test  fitted to the data. To do this, do the following:
* Fit a **fixed max depth** (`i`) decision tree regressor using *all `x_train` variables*. Also, *fix the `random state` to 1* for reproducibility.
* Register its train and test RMSEs
* Plot the train and test RMSE curves for each `i` depth  

Make sure that you explore _enough_ depth values. 

In [ ]:
# Lists to store the results
train_rmse = []
test_rmse = []

# Range of depths to be explored
tree_size = np.arange(1,20)

for i in tree_size:

    # Train the needed tree with the set depth, then measure its RMSE in train and test and store them in the previous lists
    tree_regressor = DecisionTreeRegressor(max_depth=i, random_state=1)
    tree_regressor.fit(x_train, y_train)

    y_train_predict = tree_regressor.predict(x_train)
    y_test_predict = tree_regressor.predict(x_test)

    train_rmse_value = np.sqrt(mean_squared_error(y_train, y_train_predict))
    test_rmse_value = np.sqrt(mean_squared_error(y_test, y_test_predict))

    train_rmse.append(train_rmse_value)
    test_rmse.append(test_rmse_value)

# Plot the results
plt.plot(tree_size, train_rmse, 'r*--', label="Train RMSE")
plt.plot(tree_size, test_rmse, 'b.-', label="Test RMSE")
plt.xlabel('Max Depth')
plt.ylabel('Root Mean Squared Error')
plt.legend()
plt.title('RMSE vs Tree Depth')
plt.show()


> Question: What do you see here? What depth value would you select? <br><br>
A medida que aumentamos la profundidad del árbol, el error en el conjunto de entrenamiento disminuye notablemente, lo que indica una alta varianza. Sin embargo, el error en el conjunto de prueba no muestra una reducción significativa, lo que sugiere un posible sobreajuste si max_depth es demasiado alto. Según la gráfica, el menor error en el test se encuentra aproximadamente en max_depth = 3, por lo que este valor sería el más adecuado, ya que logra un equilibrio entre el ajuste a los datos de entrenamiento y la capacidad de generalización.



Since we are not really making CV, we do not have multiple values for the train and test RMSE for each tree. Therefore, we *do not* have errorbars in the previous plot. That should raise some suspicions from your part. 

> Question: What happens if we change the `random_state` value? Are the previous results robust? <br><br> Como se observa en el gráfico, el RMSE del conjunto de prueba varía significativamente según el valor de random_state, lo que indica que el modelo es sensible a la aleatoriedad. A partir de max_depth > 10, los valores de RMSE se estabilizan, lo que sugiere que el árbol es más influenciado por cómo se dividen los datos en las primeras etapas de decisión. Para obtener un modelo más robusto, la profundidad óptima debería minimizar el RMSE en promedio en varias ejecuciones con distintas semillas. En este caso, ese valor óptimo parece estar alrededor de 3.


To answer the previous question you can try out code in the next cell. Feel free to try whatever you think is needed here.

In [ ]:
# Lists to store the results for different random states
random_states = [0, 1, 42, 100] 
train_rmse_results = []
test_rmse_results = []

# Range of depths to be explored
tree_size = np.arange(1, 20)

for rs in random_states:
    train_rmse = []
    test_rmse = []

    for i in tree_size:
        tree_regressor = DecisionTreeRegressor(max_depth=i, random_state=rs)
        tree_regressor.fit(x_train, y_train)

        y_train_predict = tree_regressor.predict(x_train)
        y_test_predict = tree_regressor.predict(x_test)

        train_rmse.append(np.sqrt(mean_squared_error(y_train, y_train_predict)))
        test_rmse.append(np.sqrt(mean_squared_error(y_test, y_test_predict)))

    train_rmse_results.append(train_rmse)
    test_rmse_results.append(test_rmse)

# Plot the results
plt.figure(figsize=(10, 6))

for i, rs in enumerate(random_states):
    plt.plot(tree_size, test_rmse_results[i], label=f"Test RMSE (random_state={rs})", linestyle="--")

plt.xlabel('Max Depth')
plt.ylabel('Root Mean Squared Error')
plt.legend()
plt.title('Effect of random_state on RMSE vs Tree Depth')
plt.show()

In order to get more acquainted with the results, check out what happens if you include less  `x_train` features. To do so, change what you need from the previous block of code and put it in the next block here.
> Question: Do you see any important changes? How do you explain this?  

In [ ]:
# Lista de valores de max_features a probar
max_features_values = [2, 4, 6, 10, 16]  

# Rango de profundidades a explorar
tree_size = np.arange(1, 20)

plt.figure(figsize=(10, 6))

for max_feat in max_features_values:
    train_rmse = []
    test_rmse = []

    for depth in tree_size:
        tree_regressor = DecisionTreeRegressor(max_depth=depth, max_features=max_feat, random_state=1)
        tree_regressor.fit(x_train, y_train)

        y_train_predict = tree_regressor.predict(x_train)
        y_test_predict = tree_regressor.predict(x_test)

        train_rmse.append(np.sqrt(mean_squared_error(y_train, y_train_predict)))
        test_rmse.append(np.sqrt(mean_squared_error(y_test, y_test_predict)))

    # Graficar los resultados
    plt.plot(tree_size, test_rmse, label=f"Test RMSE (max_features={max_feat})", linestyle="--")
    
plt.xlabel('Max Depth')
plt.ylabel('Root Mean Squared Error')
plt.legend()
plt.title('RMSE vs Tree Depth for Different max_features Values')
plt.show()

## Classification Tree

In order to complete this practical exercises with trees, we will also try out some classification trees to later do ensembles. Let us see how this works. 

First, load the `Carseat.csv` dataset from `data` (make sure to remove the NAs, as before)

In [ ]:
carseats = pd.read_csv('../data/Carseat.csv', sep = ',')
carseats = carseats.dropna()
carseats

Make it so that we have a new binary variable called `high`. This variable should be `1` when `sales` are over 8, and `0` otherwise.

In [ ]:

carseats['high'] = (carseats['Sales'] > 8).astype(int)

Convert the remaining variables to make them usable here

The variables `ShelveLoc`, `Urban` and `US` need to be converted to categorical variables to be correctly used. To that end, I suggest you use `pd.factorize` (although feel free to do as you will here...)

In [ ]:
# Your code here!
carseats['ShelveLoc'] = pd.factorize(carseats['ShelveLoc'])[0]
carseats['Urban'] = pd.factorize(carseats['Urban'])[0]
carseats['US'] = pd.factorize(carseats['US'])[0]

Now, we will employ all variables to predict the `high` value (except `Sales` and `high`, for obvious reasons). Note that we have essentially converted a _regression_ problem into a _binary classification_ one.

In [ ]:
X  =  carseats.drop(columns=['Sales', 'high'])
y =  carseats['high']

# Performn the train/test split with again 50% data for train and 50% for test 
X_train, X_test, y_train, y_test= train_test_split(X, y,test_size=0.5, random_state = 0)

Construct a decision tree classifier. To control for the depth, we will fix it to a *maximum depth of 6*. Use as impurity criteria the **Gini index**.

In [ ]:
carseats_classifier = DecisionTreeClassifier(criterion='gini', max_depth=6, random_state=0)  # TODO: Fill the NAs. Fix the random_state to 0

# Train the model with .fit
carseats_classifier.fit(X_train, y_train) 

Plot the tree (again, reuse whatever you may need here)

In [ ]:
dot_data = StringIO()
export_graphviz(carseats_classifier, 
                out_file=dot_data, 
                feature_names= X.columns, 
                filled=True, 
                class_names=None)

graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
Image(graph.create_png())

Let us now assess the quality of the tree. To that end, **represent the confusion matrix** for the test data

In [ ]:
predictions = carseats_classifier.predict(X_test)
matriz_confusion = confusion_matrix(y_test, predictions)

plt.figure(figsize=(6, 5))
sns.heatmap(matriz_confusion, annot=True, fmt='d', cmap='Blues', xticklabels=['Low', 'High'], yticklabels=['Low', 'High'])
plt.title('Confusion Matrix')
plt.xlabel('Predicted_label')
plt.ylabel('True_label')
plt.show()

> Questions: 
> * What is the **precision** of this tree?<br><br>
0.746
> * Do you consider the dataset balanced? <br><br> 
No esta muy desbalanceado, no obstante hay un mayor numero de trabajo de la clase Low que High

# Ensembles

Now we will try out some of the ensemble methods from class. Remember there is an stochastic component embedded in these for the most part, so we may not recover exactly the same results twice depending on how you implement things.

## Bagging

The decision tree models mentioned above usually suffers from high variance. **B**ootstrap **agg**regation, or **bagging** usually helps with this issue. To do bagging here, we will do it both by hand and by employing the sklearn function.

First, let's go with the *by-hand* implementation

In [ ]:
# Set the bagging parameters
n_estimators = 10  # Number of decision trees in the ensemble
max_samples = 0.8  # Proportion of samples to be used for each bootstrap sample

# Store the predictions
predictions = []


n_samples = X_train.shape[0]
bootstrap_size = int(max_samples * n_samples)

for _ in range(n_estimators):
    
    # Create a bootstrap sample
    sample_indices = np.random.choice(n_samples, bootstrap_size, replace=True)
    X_bootstrap = X_train.iloc[sample_indices]
    y_bootstrap = y_train.iloc[sample_indices]
    
    # Train a decision tree classifier on the bootstrap sample
    decision_tree = DecisionTreeClassifier()  # No restrictions
    decision_tree.fit(X_bootstrap, y_bootstrap)
    
    # Make predictions on the test set using the trained decision tree
    y_pred = decision_tree.predict(X_test)
    predictions.append(y_pred)

# Convert predictions list to a NumPy array
predictions = np.array(predictions)

# Combine predictions using majority voting (for classification)
majority_vote = mode(predictions, axis=0).mode[0]

# If this were regression, you would use averaging instead:
combined_predictions = np.mean(predictions, axis=0)

Print the confusion matrix

In [ ]:
# Your code here! Use majority_vote and y_test
clases = np.unique(y_test)
num_clases = len(clases)
matriz_confusion = np.zeros((num_clases, num_clases), dtype=int)

#Con esto, obtenemos ínidices númericos respecto a cada etiqueta según las posibles clases exixtentes
indices_reales = np.searchsorted(clases, y_test)
indices_predichos = np.searchsorted(clases, majority_vote)


for i in range(len(y_test)):
    matriz_confusion[indices_reales[i], indices_predichos[i]] += 1

print("Matriz de Confusión:")
print(matriz_confusion)
plt.figure(figsize=(6, 5))
sns.heatmap(matriz_confusion, annot=True, fmt='d', cmap='Greens', xticklabels=['Low', 'High'], yticklabels=['Low', 'High'])
plt.title('Confusion Matrix')
plt.xlabel('Predicted_label')
plt.ylabel('True_label')
plt.show()

Let's see how this is done in `sklearn`... Fit it and show the confusion matrix

In [ ]:
# Create a BaggingClassifier (fix random_state to 0)
bagging = BaggingClassifier(random_state=0)

#Train it with the training data
bagging.fit(X_train, y_train)

# Obtain the predictions
bagging_pred = bagging.predict(X_test)

#Print the confusion matrix (use the confusion_matrix function)
mat_conf = confusion_matrix(y_test, bagging_pred)
plt.figure(figsize=(6, 5))
sns.heatmap(mat_conf, annot=True, fmt='d', cmap='Oranges', xticklabels=['Low', 'High'], yticklabels=['Low', 'High'])
plt.title('Confusion Matrix')
plt.xlabel('Predicted_label')
plt.ylabel('True_label')
plt.show()

We will use the `sklearn` implementation to study the variable importance. *Make sure you understand how this is done!*

In [ ]:
feature_importances = np.mean(
    [tree.feature_importances_ for tree in bagging.estimators_], axis=0)


bagging_featureImportance = pd.DataFrame({'Feature Importance': feature_importances * 100}, index=X.columns)
bagging_featureImportance.sort_values('Feature Importance', ascending=False).plot(kind='bar', color='red')
plt.title('Feature Importance in Bagging')
plt.ylabel('Importance (%)')
plt.show()

> Question: What do you see here? <br><br>Podemos observar que la variable más significativa es Price, con una influencia superior al 30%, lo que indica que es el factor que más impacta en las predicciones del modelo. Las demás variables también contribuyen a las decisiones del modelo, aunque su relevancia disminuye progresivamente.

## Random Forest

We will also do this in the RF case. First, we will implement it by hand. Feel free to use the previous code and modify it as you may see fit to do RF here!

In [ ]:

# RF parameters
n_estimators = 10  # Number of decision trees in the forest
max_features = 0.8  # Proportion of features to consider for each split

# Train decision trees with random feature selection and make predictions
predictions = []
for _ in range(n_estimators):

    # TODO: Construct your own RF ensemble! Reuse the Bagging code and change whatever you may need here
      # Create a bootstrap sample
    sample_indices = np.random.choice(np.arange(0, X_train.shape[0], 1), size=int(X_train.shape[0]*max_samples), replace=True)# TODO
    X_bootstrap = X_train.iloc[sample_indices]# TODO 
    y_bootstrap = y_train.iloc[sample_indices]# TODO 
    
    
    # Train a decision tree classifier on the bootstrap sample
    decision_tree =   DecisionTreeClassifier(max_features="sqrt")# TOD
    decision_tree.fit(X_bootstrap, y_bootstrap)
    
    # Make predictions on the test set using the trained decision tree
    y_pred = decision_tree.predict(X_test)# TODO: Obtain the predictions for X_test
    predictions.append(y_pred)


# TODO: Finally, combine predictions using majority voting
majority_vote = []# TODO
for i in range(X_test.shape[0]):
    true_count = 0
    for t in predictions:
        if t[i]:
            true_count +=1
    if true_count >= n_estimators/2:
        majority_vote.append(True)
    else:
        majority_vote.append(False)

Print the confusion matrix

In [ ]:
# Your code here!
confusion_matrix(y_test,majority_vote)

Let us do it again with `sklearn` so you see the differences... Show the confusion matrix. In this last part there may be some differences with your run, which are due to the randomness of the classifiers constructed. Do not worry too much about it here.

In [ ]:
# Construct the RF classifier with RandomForestClassifier. Fix the random_state to 0, n_estimators to 10 and max_features to 0.8
rf = RandomForestClassifier(random_state=0,n_estimators=10,max_features=0.8)

# Train it
rf.fit(X_train, y_train)

# Predict the test values
rf_pred= rf.predict(X_test)

# Obtain the confusion matrix and print it
confusion_matrix(y_test,rf_pred)


Using again the `sklearn` implementation, we will study the feature importance

In [ ]:
rf_featureImportance= pd.DataFrame({'Feature Importance':rf.feature_importances_*100}, index= X.columns)
rf_featureImportance.sort_values('Feature Importance', ascending=False).plot(kind='bar', color='red')

> Question: Given the models thus far (simple tree, bagging and RF), which one would you choose and why? <br><br>El modelo que mejor performace tiene el de random forest y ya que podemos ver en la confusion matrix qeu los tru positives y true negatives , es decir los valores que estan bien clasificados son mayores.

## Extra: Boosting

In order to fully complete our review of the ensemble methods from class, we are missing the **Boosting method**. In order to keep matters simple, we will implement it here with `sklearn` so that you get to see what it looks like...

In [ ]:
# Import the model
from sklearn.ensemble import GradientBoostingClassifier

# Create the Boosting model
gb= GradientBoostingClassifier(n_estimators = 5000, random_state = 1, max_depth = 2)

# If you want to try it out, you can change reuse most of previous codes to run it here!

The results are pretty good! Keep in mind that this is achieved with super weak learners s.a. trees with depth 2. It is quite fast, and super easy to use with `sklearn`. We can also study the variable importance in this case.

In [ ]:
gb_featureimportance= pd.DataFrame({'Feature Importance': gb.feature_importances_*100}, index= X.columns)
gb_featureimportance.sort_values('Feature Importance', ascending=False).plot(kind='bar', color='red')

If you want to see how this can be implemented easily by hand, you can use the following code. We are using an implementation that follows a description of Boosting mode similar to the one given in the ISLR book

In [ ]:
n_estimators = 1000  # Number of decision trees in the ensemble
learning_rate = 0.1  # Learning rate for each decision tree

# Initialize the weights for the training samples
sample_weights = np.ones(len(X_train)) / len(X_train)

# Train decision trees with weighted samples and make predictions
predictions = []
for _ in range(n_estimators):

    # Train a decision tree classifier on the weighted training samples
    decision_tree = DecisionTreeClassifier(max_depth = 2)
    decision_tree.fit(X_train, y_train, sample_weight=sample_weights)

    # Make predictions on the test set using the trained decision tree
    y_pred = decision_tree.predict(X_test)
    predictions.append(y_pred)

    # Calculate error
    incorrect = (y_pred != y_test).astype(int)
    error = np.sum(sample_weights * incorrect) / np.sum(sample_weights)

    # Update sample weights
    alpha = learning_rate * np.log((1 - error) / error)
    sample_weights *= np.exp(alpha * incorrect)

# Combine predictions
# For classification, you can use weighted voting
combined_predictions = np.zeros(len(X_test))
for prediction in predictions:
    combined_predictions += prediction
